# 第五章 基于文档的问答

 - [一、导入embedding模型和向量存储组件](#一、导入embedding模型和向量存储组件)
     - [1.1 创建向量存储](#1.1-创建向量存储)
     - [1.2 使用语言模型与文档结合使用](#1.2-使用语言模型与文档结合使用)
 - [二、 如何回答我们文档的相关问题](#二、-如何回答我们文档的相关问题)


In [1]:
# set the init envs 
import os, sys
import openai
import langchain
sys.path.append('../..')

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) 


openai.api_base = os.environ["OPENAI_API_BASE"] # 换成代理，一定要加v1
openai.api_key = os.environ["OPENAI_API_KEY"]

from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI

llm = OpenAI(
    temperature=0.8,
    model_name="gpt-3.5-turbo",
)
chat = ChatOpenAI(
    temperature=0.0,
    model_name = "gpt-3.5-turbo",
)
# os.environ["LANGCHAIN_WANDB_TRACING"] = "true"
# os.environ["WANDB_PROJECT"] = "langchainPlayGround"


def get_completion(prompt, model="gpt-3.5-turbo"):
    
    messages = [{"role": "user", "content": prompt}]
    
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, 
    )
    return response.choices[0].message["content"]

from langchain.chat_models import ChatOpenAI
from langchain.llms import OpenAI

chat = ChatOpenAI(
    model_name = "gpt-3.5-turbo",
    temperature=0.2
    )
print(chat)

# llm = OpenAI(
#     model_name = "gpt-3.5-turbo",
#     temperature = 0.0
# )
# print(llm)

d:\Anaconda3\envs\LLM\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
d:\Anaconda3\envs\LLM\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
d:\Anaconda3\envs\LLM\lib\site-packages\numpy\.libs\libopenblas64__v0.3.23-246-g3d31191b-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


cache=None verbose=False callbacks=None callback_manager=None tags=None metadata=None client=<class 'openai.api_resources.chat_completion.ChatCompletion'> model_name='gpt-3.5-turbo' temperature=0.2 model_kwargs={} openai_api_key='sk-S0TKP0Sxk8H8VirL7a912cAf06314e46AfB0348eD40183B5' openai_api_base='https://openkey.cloud/v1' openai_organization='' openai_proxy='' request_timeout=None max_retries=6 streaming=False n=1 max_tokens=None tiktoken_model_name=None


### 2.2 文本向量表征模型

In [2]:
#使用OpenAIEmbedding类
from langchain.embeddings import OpenAIEmbeddings 

embedding_model = OpenAIEmbeddings() #初始化


#因为文档比较短了，
#所以这里不需要进行任何分块,
#可以直接进行向量表征
#使用初始化OpenAIEmbedding实例上的查询方法embed_query为文本创建向量表征
embed = embedding_model.embed_query("Hi my name is Harrison")

#查看得到向量表征的长度
print(len(embed))

#每个元素都是不同的数字值，组合起来就是文本的向量表征
print(embed[:5])

1536
[-0.022206753545421832, 0.006513645422377796, -0.018233178227007405, -0.039207618777269276, -0.014360198785407882]


## import some package and load the data

In [3]:
from langchain.chains import RetrievalQA #检索QA链，在文档上进行检索
from langchain.chat_models import ChatOpenAI #openai模型
from langchain.document_loaders import CSVLoader #文档加载器，采用csv格式存储
from langchain.vectorstores import DocArrayInMemorySearch #向量存储
from IPython.display import display, Markdown #在jupyter显示信息的工具

llm = chat

In [4]:
#用Pandas导入并查看数据
import pandas as pd

data = pd.read_csv('./data/OutdoorClothingCatalog_1000.csv',usecols=[1, 2])
data.head()

,name,description
0,Women's Campside Oxfords,This ultracomfortable lace-to-toe Oxford boast...
1,"Recycled Waterhog Dog Mat, Chevron Weave",Protect your floors from spills and splashing ...
2,Infant and Toddler Girls' Coastal Chill Swimsu...,"She'll love the bright colors, ruffles and exc..."
3,"Refresh Swimwear, V-Neck Tankini Contrasts",Whether you're going for a swim or heading out...
4,EcoFlex 3L Storm Pants,Our new TEK O2 technology makes our four-seaso...


### 1.2 基本文档加载器创建向量存储

In [6]:
from langchain.indexes import VectorstoreIndexCreator #导入向量存储索引创建器

loader = CSVLoader(
    file_path='./data/OutdoorClothingCatalog_1000.csv',
    encoding="utf-8"
    )

# 创建指定向量存储类
# 创建完成后，从加载器中调用, 通过文档加载器列表加载
vector_storage = VectorstoreIndexCreator(vectorstore_cls=
                                DocArrayInMemorySearch
                                ).from_loaders([loader])

print(vector_storage)

Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised RateLimitError: 当前分组负载已饱和，请稍后再试，或升级账户以提升服务质量。.
Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised RateLimitError: 当前分组负载已饱和，请稍后再试，或升级账户以提升服务质量。.
Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised RateLimitError: 当前分组负载已饱和，请稍后再试，或升级账户以提升服务质量。.
Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 8.0 seconds as it raised RateLimitError: 当前分组负载已饱和，请稍后再试，或升级账户以提升服务质量。.
Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised RateLimitError: 当前分组负载已饱和，请稍后再试，或升级账户以提升服务质量。.
Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised RateLimitError: 当前分组负载已饱和，请稍后再试，或升级账户以提升服务质量。.
Retrying langchain.embeddings.openai.embed_with_retry.<locals>._

vectorstore=<langchain.vectorstores.docarray.in_memory.DocArrayInMemorySearch object at 0x0000029BCC0C9F60>


### 1.3 查询创建的向量存储

In [7]:
input_query ="Please list all your shirts with sun protection \
in a table in markdown and summarize each one."


#使用索引查询创建一个响应，并传入这个查询
response = vector_storage.query(input_query)

#查看查询返回的内容
display(Markdown(response))

Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised RateLimitError: 当前分组负载已饱和，请稍后再试，或升级账户以提升服务质量。.
Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: 当前分组负载已饱和，请稍后再试，或升级账户以提升服务质量。.
Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: 当前分组负载已饱和，请稍后再试，或升级账户以提升服务质量。.




| Name | Description |
| --- | --- |
| Men's Tropical Plaid Short-Sleeve Shirt | UPF 50+ rated, 100% polyester, wrinkle-resistant, front and back cape venting, two front bellows pockets |
| Men's Plaid Tropic Shirt, Short-Sleeve | UPF 50+ rated, 52% polyester and 48% nylon, machine washable and dryable, front and back cape venting, two front bellows pockets |
| Men's TropicVibe Shirt, Short-Sleeve | UPF 50+ rated, 71% Nylon, 29% Polyester, 100% Polyester knit mesh, machine wash and dry, front and back cape venting, two front bellows pockets |
| Sun Shield Shirt by | UPF 50+ rated, 78% nylon, 22% Lycra Xtra Life fiber, handwash, line dry, wicks moisture, fits comfortably over swimsuit, abrasion resistant |

All four shirts provide UPF 50+ sun protection, blocking 98% of the sun's harmful rays. The Men's Tropical Plaid Short-Sleeve Shirt is made of 100% polyester and is wrinkle-resistant

## 二、 结合表征模型和向量存储

使用语言模型与文档结合使用时，语言模型一次只能使用几千个单词的信息。如果我们文档比较长，如何让语言模型回答关于其中所有内容的问题呢？我们通过几个通过向量表征（Embeddings）和向量存储(Vector Store)实现

文本表征（Embeddings)是对文本语义的向量表征，相似内容的文本具有相似的表征向量。这使我们可以在向量空间中比较文本的相似性。

向量数据库(Vector Database)用来存储文档的文本块。对于给定的文档，我们首先将其分成较小的文本块（chunks），然后获取每个小文本块的文本表征，并将这些表征储存在向量数据库中。这个流程正是创建索引(index)的过程。将文档分成小文本快的原因在于我们可能无法将整个文档传入语言模型进行处理。

索引创建完成后，我们可以使用索引来查找与传入的查询(Query)最相关的文本片段 - 首先我们为查询获取向量表征，然后我们将其与向量数据库中的所有向量进行比较并选择最相似的n个文本块，最后我们将这些相似的文本块构建提示，并输入到语言模型，从而得到最终答案。

### 2.1 导入数据

In [8]:
#创建一个文档加载器，通过csv格式加载
loader = CSVLoader(
    file_path='./data/OutdoorClothingCatalog_1000.csv',
    encoding="utf-8"
    )

docs = loader.load()

#查看单个文档，每个文档对应于CSV中的一行数据
docs[0]

Document(page_content=": 0\nname: Women's Campside Oxfords\ndescription: This ultracomfortable lace-to-toe Oxford boasts a super-soft canvas, thick cushioning, and quality construction for a broken-in feel from the first time you put them on. \n\nSize & Fit: Order regular shoe size. For half sizes not offered, order up to next whole size. \n\nSpecs: Approx. weight: 1 lb.1 oz. per pair. \n\nConstruction: Soft canvas material for a broken-in feel and look. Comfortable EVA innersole with Cleansport NXT® antimicrobial odor control. Vintage hunt, fish and camping motif on innersole. Moderate arch contour of innersole. EVA foam midsole for cushioning and support. Chain-tread-inspired molded rubber outsole with modified chain-tread pattern. Imported. \n\nQuestions? Please contact us for any inquiries.", metadata={'source': './data/OutdoorClothingCatalog_1000.csv', 'row': 0})

### 2.3 基于向量表征创建向量存储并进行查询

### 2.4 查询创建的向量存储

In [9]:
# import langchain
# (langchain.__version__)

# 将刚才创建文本向量表征(embeddings)存储在向量存储(vector store)中
# 使用DocArrayInMemorySearch类的from_documents方法来实现
# 该方法接受文档列表以及向量表征模型作为输入
vector_db = DocArrayInMemorySearch.from_documents(docs, embedding_model)


query = "Please suggest a shirt with sunblocking"

#使用上面的向量存储来查找与传入查询类似的文本，得到一个相似文档列表
docs = vector_db.similarity_search(query)

#打印返回文档的个数
print(len(docs))


#打印返回的第一个文档
print(docs[0])

# 我们可以看到一个返回了四个结果。输出的第一结果是一件关于防晒的衬衫，满足我们查询的要求：`请推荐一件防晒功能的衬衫（Please suggest a shirt with sunblocking）`

Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised RateLimitError: 当前分组负载已饱和，请稍后再试，或升级账户以提升服务质量。.
Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised RateLimitError: 当前分组负载已饱和，请稍后再试，或升级账户以提升服务质量。.
Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised RateLimitError: 当前分组负载已饱和，请稍后再试，或升级账户以提升服务质量。.
Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised RateLimitError: 当前分组负载已饱和，请稍后再试，或升级账户以提升服务质量。.


4
page_content=': 255\nname: Sun Shield Shirt by\ndescription: "Block the sun, not the fun – our high-performance sun shirt is guaranteed to protect from harmful UV rays. \n\nSize & Fit: Slightly Fitted: Softly shapes the body. Falls at hip.\n\nFabric & Care: 78% nylon, 22% Lycra Xtra Life fiber. UPF 50+ rated – the highest rated sun protection possible. Handwash, line dry.\n\nAdditional Features: Wicks moisture for quick-drying comfort. Fits comfortably over your favorite swimsuit. Abrasion resistant for season after season of wear. Imported.\n\nSun Protection That Won\'t Wear Off\nOur high-performance fabric provides SPF 50+ sun protection, blocking 98% of the sun\'s harmful rays. This fabric is recommended by The Skin Cancer Foundation as an effective UV protectant.' metadata={'source': './data/OutdoorClothingCatalog_1000.csv', 'row': 255}


### 2.5 使用向量储存回答文档的相关问题

In [10]:
input_query = "Please suggest a shirt with sunblocking"

#使用上面的向量存储来查找与传入查询类似的文本，得到一个相似文档列表
obtained_docs = vector_db.similarity_search(input_query)

#基于向量储存，创建检索器
retriever = vector_db.as_retriever() 

#导入大语言模型, 这里使用默认模型gpt-3.5-turbo会出现504服务器超时，因此使用gpt-3.5-turbo-0301
llm = ChatOpenAI(
    model_name="gpt-3.5-turbo-0301",
    temperature = 0.0
                 ) 


#合并获得的相似文档内容
result_docs = "".join([obtained_docs[i].page_content for i in range(len(obtained_docs))])  


#将合并的相似文档内容后加上问题（question）输入到 `llm.call_as_llm`中
#这里问题是：以Markdown表格的方式列出所有具有防晒功能的衬衫并总结
response = llm.call_as_llm(f"{result_docs} Question: Please list all your \
shirts with sun protection in a table in markdown and summarize each one.") 


# 在此处打印响应，我们可以看到我们得到了一个表格，正如我们所要求的那样
display(Markdown(response))



Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised RateLimitError: 当前分组负载已饱和，请稍后再试，或升级账户以提升服务质量。.
Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised RateLimitError: 当前分组负载已饱和，请稍后再试，或升级账户以提升服务质量。.
Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised RateLimitError: 当前分组负载已饱和，请稍后再试，或升级账户以提升服务质量。.
Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 8.0 seconds as it raised RateLimitError: 当前分组负载已饱和，请稍后再试，或升级账户以提升服务质量。.
Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 10.0 seconds as it raised RateLimitError: 当前分组负载已饱和，请稍后再试，或升级账户以提升服务质量。.


RateLimitError: 当前分组负载已饱和，请稍后再试，或升级账户以提升服务质量。

In [ ]:
''' 
通过LangChain链封装起来
创建一个检索QA链，对检索到的文档进行问题回答，要创建这样的链，
我们将传入几个不同的东西
1、语言模型，在最后进行文本生成
2、传入链类型，这里使用stuff，将所有文档塞入上下文并对语言模型进行一次调用
3、传入一个检索器
'''


RetrievalQAChain = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=retriever, 
    verbose=True
)


query =  "Please list all your shirts with sun protection in a table \
in markdown and summarize each one."#创建一个查询并在此查询上运行链


response = RetrievalQAChain.run(query)


display(Markdown(response))#使用 display 和 markdown 显示它



> Entering new RetrievalQA chain...


Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised RateLimitError: 当前分组负载已饱和，请稍后再试，或升级账户以提升服务质量。.
Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised RateLimitError: 当前分组负载已饱和，请稍后再试，或升级账户以提升服务质量。.
Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised RateLimitError: 当前分组负载已饱和，请稍后再试，或升级账户以提升服务质量。.
Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 8.0 seconds as it raised RateLimitError: 当前分组负载已饱和，请稍后再试，或升级账户以提升服务质量。.
Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 10.0 seconds as it raised RateLimitError: 当前分组负载已饱和，请稍后再试，或升级账户以提升服务质量。.
Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: 当前分组负载已饱和，请稍后再试，或升级账户以提升服务质量。.
Retrying langchain.chat_models.openai.Cha


> Finished chain.


| Shirt Number | Name | Description |
| --- | --- | --- |
| 618 | Men's Tropical Plaid Short-Sleeve Shirt | This shirt is made of 100% polyester and is wrinkle-resistant. It has front and back cape venting that lets in cool breezes and two front bellows pockets. It is rated UPF 50+ for superior protection from the sun's UV rays. |
| 374 | Men's Plaid Tropic Shirt, Short-Sleeve | This shirt is made with 52% polyester and 48% nylon. It is machine washable and dryable. It has front and back cape venting, two front bellows pockets, and is rated to UPF 50+. |
| 535 | Men's TropicVibe Shirt, Short-Sleeve | This shirt is made of 71% Nylon and 29% Polyester. It has front and back cape venting that lets in cool breezes and two front bellows pockets. It is rated UPF 50+ for superior protection from the sun's UV rays. |
| 255 | Sun Shield Shirt | This shirt is made of 78% nylon and 22% Lycra Xtra Life fiber. It is handwashable and line dry. It is rated UPF 50+ for superior protection from the sun's UV rays. It is abrasion-resistant and wicks moisture for quick-drying comfort. |

All of these shirts provide UPF 50+ sun protection, blocking 98% of the sun's harmful rays. They are all designed to be lightweight and comfortable in hot weather. They all have front and back cape venting that lets in cool breezes and two front bellows pockets. The Men's Tropical Plaid Short-Sleeve Shirt is made of 100% polyester and is wrinkle-resistant. The Men's Plaid Tropic Shirt, Short-Sleeve is made with 52% polyester and 48% nylon. The Men's TropicVibe Shirt, Short-Sleeve is made of 71% Nylon and 29% Polyester. The Sun Shield Shirt is made of 78% nylon and 22% Lycra Xtra Life fiber and is abrasion-resistant and wicks moisture for quick-drying comfort.

想在许多不同类型的块上执行相同类型的问答，该怎么办？之前的实验中只返回了4个文档，如果有多个文档，那么我们可以使用几种不同的方法
* Map Reduce   
将所有块与问题一起传递给语言模型，获取回复，使用另一个语言模型调用将所有单独的回复总结成最终答案，它可以在任意数量的文档上运行。可以并行处理单个问题，同时也需要更多的调用。它将所有文档视为独立的
* Refine    
用于循环许多文档，际上是迭代的，建立在先前文档的答案之上，非常适合前后因果信息并随时间逐步构建答案，依赖于先前调用的结果。它通常需要更长的时间，并且基本上需要与Map Reduce一样多的调用
* Map Re-rank   
对每个文档进行单个语言模型调用，要求它返回一个分数，选择最高分，这依赖于语言模型知道分数应该是什么，需要告诉它，如果它与文档相关，则应该是高分，并在那里精细调整说明，可以批量处理它们相对较快，但是更加昂贵
* Stuff    
将所有内容组合成一个文档